# Bài tập lập trình: SGD với dữ liệu thực tế

```
ProtonX - TensorFlow Class
```

### Hướng dẫn làm bài
- Trong bài tập này bạn sẽ sử dụng Python 3.


### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**


Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO` và thay thế các vị trí `None`.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó.

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Phải có log quá trình training
* Mean Square Error nhỏ hơn hoặc bằng 2535885073

In [39]:
import tensorflow as tf
import pandas as pd

In [40]:
train_df = pd.read_csv('/content/sample_data/california_housing_train.csv')
val_df = pd.read_csv('/content/sample_data/california_housing_test.csv')

In [41]:
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


### Chuẩn hóa dữ liệu

In [42]:
X_train = train_df.drop('median_house_value', axis = 1)
y_train = train_df['median_house_value']

In [43]:
import pandas as pd
from sklearn import preprocessing

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

column_names_to_normalize = ['longitude', 'latitude', 'total_rooms', 'total_bedrooms', 'population', 'households']

x = X_train[column_names_to_normalize].values

x_scaled = min_max_scaler.fit_transform(x)

normalized_features = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = X_train.index)

X_train[column_names_to_normalize] = normalized_features

In [44]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

column_names_to_standardize = ['housing_median_age', 'median_income']

x = X_train[column_names_to_standardize].values

x_scaled = standard_scaler.fit_transform(x)

standardized_features = pd.DataFrame(x_scaled, columns=column_names_to_standardize, index = X_train.index)

X_train[column_names_to_standardize] = standardized_features

In [45]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,1.000000,0.175345,-1.079671,0.147885,0.198945,0.028364,0.077454,-1.252543
1,0.984064,0.197662,-0.761872,0.201608,0.294848,0.031559,0.075974,-1.081483
2,0.975100,0.122210,-0.920772,0.018927,0.026847,0.009249,0.019076,-1.170105
3,0.974104,0.116897,-1.159121,0.039515,0.052142,0.014350,0.037000,-0.362600
4,0.974104,0.109458,-0.682422,0.038276,0.050435,0.017405,0.042921,-1.026454
...,...,...,...,...,...,...,...,...
16995,0.008964,0.854410,1.859971,0.058389,0.060987,0.025337,0.060516,-0.799999
16996,0.007968,0.866100,0.588774,0.061869,0.081782,0.033381,0.076303,-0.715727
16997,0.004980,0.988310,-0.920772,0.070515,0.082247,0.034782,0.074823,-0.446663
16998,0.004980,0.984060,-0.761872,0.070384,0.085506,0.036296,0.078441,-0.997787


### Tiến hành training

In [46]:
import numpy as np
m, n = X_train.shape
ones = np.ones((m, 1))
X_train = np.concatenate((X_train, ones), axis=1)


In [47]:
# Tạo tham số theta (9, ) -> (9, 1)
theta = np.random.normal(size=X_train.shape[1]).reshape((X_train.shape[1], 1))

X_train.dot(theta).shape

# Chuyển chiều từ (17000, ) -> (17000, 1)
y_train = y_train.values.reshape(-1, 1)


In [48]:
X_train.shape, theta.shape, y_train.shape

((17000, 9), (9, 1), (17000, 1))

In [49]:
X_train[0].shape

(9,)

In [50]:
theta.shape

(9, 1)

In [51]:
X_train[0].dot(theta).shape

(1,)

In [52]:
print(X_train.shape)
print(y_train.shape)

(17000, 9)
(17000, 1)


### Xây dựng mô hình SGD tại đây

In [53]:
from sklearn.utils import shuffle

# learning_rate = 0.01

def hypo(x, theta):
  # TODO 1: Tính toán hàm giả thiết
  return x.dot(theta)

def cost(X, y, theta, m):
  # TODO 2: Tính giá trị mất mát trên toàn bộ tập dữ liệu
  return 1/(2*m)*np.sum((X.dot(theta)-y)**2)

def SGD(X, y, theta, learning_rate, epochs):
  m = len(X)
  for i in range(epochs):
    # TODO 3: Xáo dữ liệu
    X, y = shuffle(X,y)
    # TODO 4: Cập nhật bộ tham số
    for k in range(m):
      error = hypo(X[k], theta) - y[k]
      for j in range(len(theta)):
        theta[j] = theta[j] - (learning_rate* error * X[k][j])
    if i % 10 == 0:
      print("epoch {} loss is {}".format(i,cost(X, y, theta, m)))

### Tiến hành Training

In [54]:
epochs = 100
learning_rate = 0.01
SGD(X_train, y_train, theta, learning_rate, epochs)

epoch 0 loss is 3093923560.068343
epoch 10 loss is 2565111820.055456
epoch 20 loss is 2572124523.4086914
epoch 30 loss is 2465937614.4592643
epoch 40 loss is 2461258529.319396
epoch 50 loss is 2491301251.773624
epoch 60 loss is 2425669147.1059637
epoch 70 loss is 2444546229.504558
epoch 80 loss is 2436024521.899945
epoch 90 loss is 2555806185.5317516


Kiểm tra Model hiện tại với tập test

In [55]:
test_df = pd.read_csv('/content/sample_data/california_housing_test.csv')
X_test = test_df.drop('median_house_value', axis = 1)
y_test = test_df['median_house_value']

# TODO 5: Lấy các cột cần chuẩn hóa
column_names_to_normalize = ['longitude', 'latitude', 'total_rooms', 'total_bedrooms', 'population', 'households']
# TODO 6: Sử dụng min và max của X_train để chuẩn hóa cho X_test
x_test = X_test[column_names_to_normalize].values
x_test_cols_scaled = min_max_scaler.transform(x_test)

# TODO 7: Tạo dataframe mới với các cột này
normalized_test_features = pd.DataFrame(x_test_cols_scaled, columns=column_names_to_normalize, index = X_test.index)

# TODO 8: Gán lại cho bảng ban đầu
X_test[column_names_to_normalize] = normalized_test_features

In [56]:
from sklearn.preprocessing import StandardScaler

column_names_to_standardize = ['housing_median_age', 'median_income']

# TODO 9: Chuẩn hóa các cột về phân phối chuẩn đơn vị
x_test_cols = X_test[column_names_to_standardize].values
x_test_cols_scaled = standard_scaler.transform(x_test_cols)
standardized_test_features = pd.DataFrame(x_test_cols_scaled, columns=column_names_to_standardize, index = X_test.index)
X_test[column_names_to_standardize] = standardized_test_features



In [57]:
len(X_test)

3000

In [58]:
print(X_test.shape)
print(y_test.shape)

(3000, 8)
(3000,)


In [59]:
import numpy as np
m_t, n_t = X_test.shape
ones_t = np.ones((m_t, 1))

In [60]:
# TODO 10: Tính mất mát trên bộ test
X_test = np.concatenate((X_test, ones_t), axis=1)
y_test = y_test.values.reshape(-1, 1)
cost(X_test, y_test, theta, len(X_test))

2501004752.5520453

Kết quả mong đợi: mất mát nhỏ hơn hoặc bằng `2535885073`